In [156]:
import torch
import torch.nn as nn
# import pandas as pd
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import random
import re
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os

#from nltk.tokenize import word_tokenize
from collections import Counter

# from ...src.utils.wiki_tokens.wiki_cleaner import getMorphemeList
# import ../src/utiils/wiki_token/wiki_cleaner

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [155]:
with open("../tokens/tokens.json", "r") as file:
    tokenizer =json.load(file)

In [157]:
with open("text", "r") as file:
    wiki = file.read()

In [ ]:

# def getMorphemeList(text):
#   text = text.lower()
#   text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#   morpheme_list = word_tokenize(text)
  
#   # Count occurrences of each word
#   word_counts = Counter(morpheme_list)
  
#   # Filter out words that only appear once
#   morpheme_list = [word for word in morpheme_list if word_counts[word] > 1]
  
#   return morpheme_list


# words = getMorphemeList(wiki)

: 

In [158]:
words = wiki.split(sep=" ")
window = 2
print (words[0])
words.pop(0)

tokenized_wiki = []

for word in words:
    if word in tokenizer:
        tokenized_wiki.append(tokenizer[word])
        

In [159]:
print (tokenized_wiki[0:100])

[1247, 1959, 1, 32287, 21917, 150, 12201, 35002, 644, 10085, 36322, 5821, 32413, 21917, 32413, 27187, 1271, 32413, 27932, 7746, 21917, 32413, 27187, 36011, 32413, 32287, 17153, 30730, 35002, 15956, 1, 23224, 35832, 32827, 8643, 1554, 324, 32408, 35002, 35439, 32827, 8704, 32413, 22197, 21917, 29756, 17216, 1000, 3013, 31899, 34900, 1959, 1, 24521, 17881, 4462, 28522, 8320, 32413, 36306, 1247, 17153, 8616, 12918, 32413, 36225, 27639, 5482, 17712, 1247, 1959, 1, 24295, 23667, 17153, 32413, 3067, 32408, 1814, 34563, 1271, 29043, 2925, 1019, 32480, 1814, 21917, 35972, 32569, 1247, 1000, 32827, 26650, 29746, 32408, 531, 32413, 10412, 21917, 2330]


In [160]:
class embed_train_dataset(Dataset):
    def __init__(self, words):
        self.data = words
    
    def __len__(self):
        return len(self.data)-4
    
    def __getitem__(self, idx):
        idx = idx+2       
        sent = self.data[max(0,idx-window):min(idx+window+1,len(words))]    
        if len(sent) > 1:
            rand_idx = random.randint(0,len(sent)-1)
            target = sent[rand_idx]
            del sent[rand_idx]
            #print (sent)
            tokenized = torch.tensor(sent)

            
            return tokenized, torch.tensor(target)
        

In [135]:
dataset = embed_train_dataset(tokenized_wiki)
dataloader = DataLoader(dataset, batch_size=1,shuffle=True)

for data in dataloader:
    print (data)
    break



[17231, 12917, 22018, 17215]
[tensor([[17231, 12917, 22018, 17215]]), tensor([32412])]


In [167]:
class CBOW(torch.nn.Module):
    def __init__(self):
        super(CBOW, self).__init__()
        self.embeddings = torch.nn.Embedding(num_embeddings = len(tokenizer), embedding_dim = 200)
        self.linear = torch.nn.Linear(in_features = 200, out_features = len(tokenizer))
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.mean(dim=1)
        out = self.linear(embeds)
        #probs =  F.log_softmax(out, dim=1)
        return out

In [174]:
def train_loop():
    number_epochs = 5

    #train_wiki, val_wiki = train_test_split(words)
    os.makedirs("checkpoints", exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    dataset = embed_train_dataset(tokenized_wiki)
    dataloader = DataLoader(dataset, batch_size=128,shuffle=True)
    

    model = CBOW().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_loss = 100000000000000.0
    for epoch in range(number_epochs):
        model.train()
        epoch_loss = 0.0
        for X,Y in tqdm(dataloader):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = F.cross_entropy(pred,Y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
            #print (loss.item())
        epoch_loss = epoch_loss/len(dataloader)
        print(f"Epoch: {epoch}/{number_epochs}, loss: {epoch_loss} ")
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            save_path = f"checkpoints/storm_epoch_{epoch+1}.pt"
            torch.save(model.state_dict(), save_path)
            print(f"Model improved. Saved to: {save_path}")





In [173]:
train_loop()

  0%|          | 9/80028 [00:00<1:38:16, 13.57it/s] 

10.54477596282959
10.51152515411377
10.531922340393066
10.4627046585083
10.455495834350586
10.49563980102539
10.45979118347168
10.42750358581543
10.394074440002441
10.407992362976074
10.35119342803955
10.415891647338867
10.303583145141602
10.255722045898438
10.309813499450684
10.3588285446167


  0%|          | 26/80028 [00:01<34:58, 38.13it/s] 

10.30722713470459
10.222648620605469
10.182543754577637
10.214749336242676
10.21517562866211
10.178634643554688
10.23403263092041
10.129247665405273
10.084440231323242
10.125654220581055
10.114340782165527
10.072029113769531
9.943923950195312
10.005247116088867
9.872567176818848
10.01550579071045
9.973922729492188
9.854177474975586


  0%|          | 44/80028 [00:01<23:02, 57.84it/s]

...

6.140798568725586
6.262440204620361
6.921427249908447
6.866779327392578
6.427647113800049
6.592676639556885
6.829952239990234
7.094641208648682
6.710075378417969
6.413084983825684
6.364757537841797
6.428272724151611
6.71788215637207
6.536314487457275
7.044521331787109
6.932684421539307
6.749797821044922


  2%|▏         | 1304/80028 [00:17<17:40, 74.27it/s]

6.161960124969482
6.604086875915527
6.541348457336426
5.720381259918213
6.521249771118164
6.937853813171387
6.704490661621094
6.386988162994385
6.283684253692627
6.855344295501709
6.703310966491699
6.17993688583374
6.206014633178711
6.13571834564209
7.127386093139648


  2%|▏         | 1305/80028 [00:17<17:27, 75.15it/s]


KeyboardInterrupt: 